In [ ]:
# Stage 0: Create environment
! pip install graphviz
! pip install topologicpy --upgrade

import topologicpy

from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Graph import Graph
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Matrix import Matrix
from topologicpy.Helper import Helper


In [ ]:
import os
import time
from topologicpy.Topology import Topology
from pathlib import Path

def _is_break(s: str) -> bool:
    return isinstance(s, str) and s.strip().lower() in {"break", "q", "quit", "exit"}

def pick_obj_folder() -> Path:
    try:
        import tkinter as tk
        from tkinter import filedialog
        tk.Tk().withdraw()
        chosen = filedialog.askdirectory(title="Select folder containing OBJ files")
        if chosen:
            p = Path(chosen)
            if p.is_dir():
                return p
    except Exception:
        pass
    while True:
        raw = input('Enter path to folder containing OBJ files (or type "break" to cancel): ').strip().strip('"')
        if _is_break(raw):
            raise KeyboardInterrupt("User cancelled folder selection.")
        p = Path(raw)
        if p.is_dir():
            return p
        print("❗ Not a valid folder. Please try again.")

# Ask at runtime instead of hard-coding:
OBJ_DIR = pick_obj_folder()
print(f"[Folder] Using OBJ directory: {OBJ_DIR}")

def import_obj_files(obj_dir):
    # Collect and sort .obj files (case-insensitive) for stable ordering
    obj_files = sorted([f for f in os.listdir(obj_dir) if f.lower().endswith('.obj')], key=str.lower)

    topologies = []
    for i, obj_file in enumerate(obj_files):
        obj_path = os.path.join(obj_dir, obj_file)
        topology = Topology.ByOBJPath(
            objPath=obj_path,
            defaultColor=[255, 255, 255],
            defaultOpacity=0.5,
            transposeAxes=True,
            removeCoplanarFaces=False,
            selfMerge=False,
            mantissa=6,
            tolerance=0.0001
        )
        topologies.append(topology)
        globals()[f"model_{i+1}"] = topology
        # Optional: echo each import line with index prefix as well
        pad = max(2, len(str(len(obj_files))))
        print(f"[{(i+1):0{pad}d}] Imported: {obj_file} as model_{i+1}")

    # Final summary with [01], [02], [03] ... prefix
    print("OBJ models imported:")
    pad = max(2, len(str(len(obj_files))))
    for i, obj_file in enumerate(obj_files):
        print(f"[{(i+1):0{pad}d}] model_{i+1} = {obj_file}")
    return topologies

def watch_obj_directory(obj_dir, interval=5):
    seen_files = set()
    while True:
        current_files = set(f for f in os.listdir(obj_dir) if f.lower().endswith('.obj'))
        new_files = current_files - seen_files
        if new_files:
            print("New OBJ files detected. Importing...")
            import_obj_files(obj_dir)
            seen_files = current_files
        elif current_files == seen_files and len(current_files) > 0:
            print("All OBJ files have been uploaded. Stopping watcher.")
            break
        time.sleep(interval)

if __name__ == "__main__":
    print("Watching OBJ directory for new files...")
    watch_obj_directory(OBJ_DIR)


In [ ]:
from datetime import datetime
import os

# Root folder where all exports go
EXPORT_ROOT = r"C:\Users\user\Downloads\Topologic_Graph_Exports"

def _ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True)
    return path

def _make_new_export_dir(root: str = EXPORT_ROOT, prefix: str = "run_") -> str:
    _ensure_dir(root)
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")  # microsecond to avoid collisions
    new_dir = os.path.join(root, f"{prefix}{stamp}")
    os.makedirs(new_dir, exist_ok=False)
    return new_dir

# Initial export folder for the first CSV export of this session
current_export_dir = _make_new_export_dir()
print(f"[Init] First export folder: {current_export_dir}")


In [ ]:
# Display imported models and group selected ones
print("Imported OBJ models:")
obj_files = [f for f in os.listdir(OBJ_DIR) if f.lower().endswith(".obj")]
model_vars = sorted(
    [n for n in globals() if n.startswith("model_") and n.split("_")[1].isdigit()],
    key=lambda n: int(n.split("_")[1]),
)
model_count = max(len(model_vars), len(obj_files))

for i in range(1, model_count + 1):
    fname = obj_files[i - 1] if i - 1 < len(obj_files) else "(no corresponding OBJ file)"
    print(f"[{i:02d}] model_{i} = {fname}")

selected = input("Enter the model numbers to group (comma-separated, e.g., 1,3,5): ")
idx = sorted({int(s) for s in selected.replace(" ", "").split(",") if s.isdigit() and 1 <= int(s) <= model_count})
names = [f"model_{i}" for i in idx]

grouped = [globals()[n] for n in names if n in globals()]
missing = [n for n in names if n not in globals()]
group_name = input("Enter a name for this group: ").strip()

groups = globals().get("groups", {})
groups[group_name] = names
globals()["groups"] = groups
globals()[group_name] = grouped

print(f"Grouped models as a list named '{group_name}':\n{grouped}")
if missing:
    print("Note: These variables were not found and were skipped:", ", ".join(missing))

Topology.Show(globals()[group_name])

In [ ]:
# Loop through each cluster in the grouped_models list and process them
models = []
for group in grouped:
    for cluster in group:
        faces = Topology.Faces(cluster)
        faces = Helper.Flatten(faces)
        model = Cell.ByFaces(faces)
        models.append(model)

print("List of models created from grouped_models:")
print(models)

Topology.Show(models) #add [0/1] after models to see ther spesific model

a1 = Topology.IsInstance(models[1], type="Cell") # checking if the first object is an instance of Cell
print(a1)

In [ ]:
models = CellComplex. ByCells(models)
Topology.Show(models)
print(models) #printing the object

cells = Topology.Cells(models)
if cells is None:
	print("No cells found.")
else:
	print(len(cells))

In [ ]:
for i in range(len(cells)):
    value = input(f"Enter the value for cell {i} (key is 'type'): ")
    d = Dictionary.ByKeyValue("type", value)
    cells[i] = Topology.SetDictionary(cells[i], d)

In [ ]:
faces = Topology.Faces(models)
for face in faces:
    parents = Topology.SuperTopologies(face, hostTopology=models, topologyType="cell")
    if len(parents) == 2:
        d1 = Topology.Dictionary(parents[0])
        d2 = Topology.Dictionary(parents[1])
        value1 = Dictionary.ValueAtKey(d1, "type")
        value2 = Dictionary.ValueAtKey(d2, "type")
        value = value1+"_"+value2
        d3 = Dictionary.ByKeyValue("type", value)
        face = Topology.SetDictionary(face, d3)
    else:
        d3 = Dictionary.ByKeyValue("type", "exterior_face")
        face = Topology.SetDictionary(face, d3)

In [ ]:
# Ask user for boolean input for viaSharedTopologies and toExteriorTopologies
via_shared = input("Enter True or False for viaSharedTopologies: ").strip().lower() == "true"
to_exterior = input("Enter True or False for toExteriorTopologies: ").strip().lower() == "true"

g = Graph.ByTopology(models, viaSharedTopologies=via_shared, toExteriorTopologies=to_exterior)
Topology.Show(g, models)

verts = Graph.Vertices(g)
print(len(verts))
for v in verts:
    d = Topology.Dictionary(v)
    print(Dictionary.Keys(d), Dictionary.Values(d))

flat_g = Graph.Reshape(g)
Topology.Show(flat_g)

#Graph.ExportToCSV(g, (r"C:\Users\user\Downloads\test_1"), 0, overwrite=True, graphLabelHeader="label_id", edgeLabelHeader="label_id", nodeLabelHeader="label_id")

In [ ]:
def _is_break(s: str) -> bool:
    return isinstance(s, str) and s.strip().lower() in {"break", "q", "quit", "exit"}

if "groups" not in globals():
    groups = {}

def get_ungrouped_models(model_count, groups):
    grouped_indices = set()
    for v in groups.values():
        for m in v:
            idx = int(m.split("_")[1])
            grouped_indices.add(idx)
    return [i for i in range(1, model_count+1) if i not in grouped_indices]

def _cleanup_orphans(obj_files, groups):
    """
    Remove globals model_<n> that don't have a corresponding OBJ file *by index*
    and also scrub them from `groups`. This keeps state consistent.
    """
    max_idx = len(obj_files)
    removed = []

    # 1) Remove any model_<n> where n exceeds current file count OR is None
    for name in list(globals()):
        if name.startswith("model_"):
            try:
                idx = int(name.split("_")[1])
            except (IndexError, ValueError):
                continue
            if idx > max_idx or globals()[name] is None:
                globals().pop(name, None)
                removed.append(idx)

    # 2) Remove any model_<n> entries from groups that were purged
    if removed:
        removed_set = {f"model_{i}" for i in removed}
        for k in list(groups.keys()):
            groups[k] = [m for m in groups[k] if m not in removed_set]
            if not groups[k]:
                # drop empty group
                groups.pop(k, None)

    return sorted(removed)

# ===================== MAIN LOOP =====================
while True:
    print("\nImported OBJ models:")
    obj_files = [f for f in os.listdir(OBJ_DIR) if f.lower().endswith('.obj')]

    # NEW: purge orphaned models that have no corresponding OBJ file
    purged = _cleanup_orphans(obj_files, groups)
    if purged:
        print(f"Removed models without OBJ: {[f'model_{i}' for i in purged]}")

    model_count = len([name for name in globals() if name.startswith("model_")])

    # Get ungrouped model indices, then filter to only those with an OBJ
    ungrouped = [i for i in get_ungrouped_models(model_count, groups) if i-1 < len(obj_files)]
    if not ungrouped:
        print("All models have been grouped and processed.")
        break

    for idx in ungrouped:
        print(f"[{idx:02d}] model_{idx} = {obj_files[idx-1]}")

    # ---- selection prompt (supports 'break') ----
    selected_raw = input("Enter the model numbers to group (comma-separated, e.g., 1,3,5) or type 'break' to stop: ").strip()
    if _is_break(selected_raw):
        print("Stopping by user request.")
        break

    selected_indices = []
    for x in selected_raw.split(","):
        x = x.strip()
        if x.isdigit():
            xi = int(x)
            if (xi in ungrouped) and (xi not in selected_indices):
                selected_indices.append(xi)

    if not selected_indices:
        print("No valid ungrouped model numbers selected. Try again.")
        continue

    grouped_models = [globals()[f"model_{i}"] for i in selected_indices]

    # ---- group name prompt (supports 'break') ----
    group_name = input("Enter a name for this group (or type 'break' to stop): ").strip()
    if _is_break(group_name):
        print("Stopping by user request.")
        break

    groups[group_name] = [f"model_{i}" for i in selected_indices]
    print(f"Grouped models as a list named '{group_name}':")
    print(grouped_models)
    globals()[group_name] = grouped_models


    # ---- Process grouped_models as in previous cells ----
    STOP_ALL = False  # sentinel to exit outer loop if user types 'break' mid-process

    models = []
    for group in grouped_models:
        for cluster in group:
            faces = Topology.Faces(cluster)
            faces = Helper.Flatten(faces)
            model = Cell.ByFaces(faces)
            models.append(model)

    models = CellComplex.ByCells(models)
    cells = Topology.Cells(models)
    if cells is None:
        print("No cells found.")
        continue

    # ---- per-cell dictionary assignment (supports 'break') ----
    for i in range(len(cells)):
        value = input(f"Enter the value for cell {i} (key is 'type') or type 'break' to stop: ").strip()
        if _is_break(value):
            print("Stopping by user request.")
            STOP_ALL = True
            break
        d = Dictionary.ByKeyValue("type", value)
        cells[i] = Topology.SetDictionary(cells[i], d)

    if STOP_ALL:
        break

    # ---- face labelling ----
    faces = Topology.Faces(models)
    for face in faces:
        parents = Topology.SuperTopologies(face, hostTopology=models, topologyType="cell")
        if len(parents) == 2:
            d1 = Topology.Dictionary(parents[0])
            d2 = Topology.Dictionary(parents[1])
            value1 = Dictionary.ValueAtKey(d1, "type")
            value2 = Dictionary.ValueAtKey(d2, "type")
            value = f"{value1}_{value2}"
            d3 = Dictionary.ByKeyValue("type", value)
            face = Topology.SetDictionary(face, d3)
        else:
            d3 = Dictionary.ByKeyValue("type", "exterior_face")
            face = Topology.SetDictionary(face, d3)

    # ---- graph flags (support 'break') ----
    via_shared_raw = input("Enter True or False for viaSharedTopologies (or type 'break' to stop): ").strip()
    if _is_break(via_shared_raw):
        print("Stopping by user request.")
        break
    to_exterior_raw = input("Enter True or False for toExteriorTopologies (or type 'break' to stop): ").strip()
    if _is_break(to_exterior_raw):
        print("Stopping by user request.")
        break

    via_shared = via_shared_raw.lower() == "true"
    to_exterior = to_exterior_raw.lower() == "true"

    g = Graph.ByTopology(models, viaSharedTopologies=via_shared, toExteriorTopologies=to_exterior)
    Topology.Show(g, models)
    print(len(verts))
    for v in verts:
        d = Topology.Dictionary(v)
        print(Dictionary.Keys(d), Dictionary.Values(d))

    flat_g = Graph.Reshape(g)
    Topology.Show(flat_g)
    # ---- export CSV to the current folder ----
_graph_export_path = _ensure_dir(current_export_dir)
Graph.ExportToCSV(
    g,
    _graph_export_path,
    0,
    overwrite=True,
    graphLabelHeader="label_id",
    edgeLabelHeader="label_id",
    nodeLabelHeader="label_id"
)
print(f"[Exported] CSV files written to: {_graph_export_path}")

# ---- immediately create the *next* empty folder for the next export ----
current_export_dir = _make_new_export_dir()
print(f"[Prepared] Next export folder: {current_export_dir}")




In [ ]:
print("Created Group Names:")
for i, group_name in enumerate(groups, start=1):
    print(f"[{i:02}] - {group_name}")


def show_model_by_name():
    name = input("Enter the model variable name (e.g., Grouped Name): ").strip()
    obj = globals().get(name, None)
    if obj is None:
        raise NameError(f"No variable named '{name}' found in the global scope.")
    # Optionally: sanity check it's a Topology (skip if not needed)
    try:
        Topology.Show(obj)
        print(f"Displayed: {name}")
    except Exception as e:
        raise TypeError(f"'{name}' is not a valid TopologicPy Topology or cannot be shown. Details: {e}")

show_model_by_name()


In [ ]:
# ===================== GRAPH COMPARISON UTILITIES =====================
import os, csv, re
from datetime import datetime

# Reuse your break helper if present; else define here
try:
    _is_break
except NameError:
    def _is_break(s: str) -> bool:
        return isinstance(s, str) and s.strip().lower() in {"break", "q", "quit", "exit"}

# Root for exports (same as you used for Graph.ExportToCSV)
try:
    EXPORT_ROOT
except NameError:
    EXPORT_ROOT = r"C:\Users\user\Downloads\Topologic_Graph_Exports"

def _ensure_dir(path: str) -> str:
    os.makedirs(path, exist_ok=True)
    return path

def _list_export_runs(root: str):
    if not os.path.isdir(root):
        return []
    # Only directories; ignore comparison_results
    runs = [d for d in os.listdir(root)
            if os.path.isdir(os.path.join(root, d)) and d.lower() != "comparison_results"]
    runs.sort()
    return runs

def _find_csvs(folder: str):
    """Heuristically pick node/edge CSVs from a folder."""
    files = [f for f in os.listdir(folder) if f.lower().endswith(".csv")]
    nodes_csv = None
    edges_csv = None
    # Prefer explicit names
    for f in files:
        fl = f.lower()
        if "node" in fl and nodes_csv is None:
            nodes_csv = f
        if "edge" in fl and edges_csv is None:
            edges_csv = f
    # Fallback: first/second csv if ambiguous
    if nodes_csv is None and files:
        nodes_csv = files[0]
    if edges_csv is None and len(files) >= 2:
        # pick the other one
        others = [f for f in files if f != nodes_csv]
        edges_csv = others[0] if others else None
    return (os.path.join(folder, nodes_csv) if nodes_csv else None,
            os.path.join(folder, edges_csv) if edges_csv else None)

def _read_csv(path: str):
    with open(path, newline="", encoding="utf-8-sig") as fp:
        rdr = csv.DictReader(fp)
        rows = [dict(r) for r in rdr]
        headers = rdr.fieldnames or []
    return headers, rows

def _detect_node_id_header(headers):
    candidates = [
        "id", "node_id", "node", "label_id", "node_label", "n", "index", "uid"
    ]
    hlow = [h.lower() for h in headers]
    for c in candidates:
        if c in hlow:
            return headers[hlow.index(c)]
    # fallback = first column
    return headers[0] if headers else None

def _detect_edge_uv_headers(headers):
    """Return (u_col, v_col, label_col or None)."""
    cand_u = ["u","source","from","start","node_u","id1","a","src","tail","head_1","s"]
    cand_v = ["v","target","to","end","node_v","id2","b","dst","head","head_2","t"]
    cand_lbl = ["label_id","label","edge_label","type","class"]
    hlow = [h.lower() for h in headers]

    u_col = v_col = lbl_col = None

    for c in cand_u:
        if c in hlow:
            u_col = headers[hlow.index(c)]
            break
    for c in cand_v:
        if c in hlow:
            v_col = headers[hlow.index(c)]
            break
    for c in cand_lbl:
        if c in hlow:
            lbl_col = headers[hlow.index(c)]
            break

    # fallback to first two columns if not found
    if (u_col is None or v_col is None) and len(headers) >= 2:
        u_col = u_col or headers[0]
        v_col = v_col or headers[1]

    return u_col, v_col, lbl_col

def _safe_id(x):
    """Normalise IDs to strings without trailing/leading spaces for stable comparison."""
    if x is None:
        return ""
    return str(x).strip()

def _load_graph_from_folder(folder: str):
    """Load graph as simple python sets/dicts from a CSV folder."""
    nodes_csv, edges_csv = _find_csvs(folder)
    if not nodes_csv or not os.path.exists(nodes_csv):
        raise FileNotFoundError(f"No node CSV found in: {folder}")
    if not edges_csv or not os.path.exists(edges_csv):
        raise FileNotFoundError(f"No edge CSV found in: {folder}")

    # Nodes
    n_headers, n_rows = _read_csv(nodes_csv)
    nid_col = _detect_node_id_header(n_headers)
    if nid_col is None:
        raise RuntimeError(f"Cannot detect node id column in: {nodes_csv}")

    nodes = set()
    node_attrs = {}  # id -> {attrs…}
    for r in n_rows:
        nid = _safe_id(r.get(nid_col))
        if not nid:
            continue
        nodes.add(nid)
        # store all attributes
        node_attrs[nid] = {k: r.get(k) for k in n_headers}

    # Edges
    e_headers, e_rows = _read_csv(edges_csv)
    u_col, v_col, e_label_col = _detect_edge_uv_headers(e_headers)
    if u_col is None or v_col is None:
        raise RuntimeError(f"Cannot detect edge endpoints in: {edges_csv}")

    # treat edges as undirected for set operations (sort endpoints)
    def _edge_key(a, b):
        a, b = _safe_id(a), _safe_id(b)
        return tuple(sorted((a, b)))

    edges = set()
    edge_attrs = {}  # (u,v) -> {attrs…}
    for r in e_rows:
        u = r.get(u_col)
        v = r.get(v_col)
        if u is None or v is None:
            continue
        ek = _edge_key(u, v)
        edges.add(ek)
        edge_attrs[ek] = {k: r.get(k) for k in e_headers}

    return {
        "nodes": nodes,
        "edges": edges,
        "node_attrs": node_attrs,
        "edge_attrs": edge_attrs,
        "nid_col": nid_col,
        "u_col": u_col,
        "v_col": v_col,
        "node_headers": n_headers,
        "edge_headers": e_headers
    }

def _compare_graphs(GA, GB):
    # Node/edge sets
    A_nodes, B_nodes = GA["nodes"], GB["nodes"]
    A_edges, B_edges = GA["edges"], GB["edges"]

    common_nodes = A_nodes & B_nodes
    onlyA_nodes = A_nodes - B_nodes
    onlyB_nodes = B_nodes - A_nodes

    common_edges = A_edges & B_edges
    onlyA_edges = A_edges - B_edges
    onlyB_edges = B_edges - A_edges

    # Attribute diffs for nodes present in both
    node_attr_diffs = []
    for nid in common_nodes:
        a = GA["node_attrs"].get(nid, {})
        b = GB["node_attrs"].get(nid, {})
        # Compare intersection of keys
        shared_keys = set(a.keys()) | set(b.keys())
        diffs = {k: (a.get(k), b.get(k)) for k in shared_keys if (a.get(k) != b.get(k))}
        if diffs:
            node_attr_diffs.append({"node_id": nid, "diffs": diffs})

    # Attribute diffs for edges present in both
    edge_attr_diffs = []
    for e in common_edges:
        a = GA["edge_attrs"].get(e, {})
        b = GB["edge_attrs"].get(e, {})
        shared_keys = set(a.keys()) | set(b.keys())
        diffs = {k: (a.get(k), b.get(k)) for k in shared_keys if (a.get(k) != b.get(k))}
        if diffs:
            edge_attr_diffs.append({"edge": e, "diffs": diffs})

    return {
        "common_nodes": common_nodes,
        "onlyA_nodes": onlyA_nodes,
        "onlyB_nodes": onlyB_nodes,
        "common_edges": common_edges,
        "onlyA_edges": onlyA_edges,
        "onlyB_edges": onlyB_edges,
        "node_attr_diffs": node_attr_diffs,
        "edge_attr_diffs": edge_attr_diffs
    }

def _write_csv(path, headers, rows):
    _ensure_dir(os.path.dirname(path))
    with open(path, "w", newline="", encoding="utf-8") as fp:
        wr = csv.DictWriter(fp, fieldnames=headers)
        wr.writeheader()
        for r in rows:
            wr.writerow(r)

def _save_comparison(root, runA, runB, GA, GB, cmpres):
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    out_root = _ensure_dir(os.path.join(root, "comparison_results", f"{runA}_VS_{runB}_{ts}"))

    # Nodes
    _write_csv(
        os.path.join(out_root, "nodes_common.csv"),
        ["node_id"],
        [{"node_id": n} for n in sorted(cmpres["common_nodes"])]
    )
    _write_csv(
        os.path.join(out_root, f"nodes_only_{runA}.csv"),
        ["node_id"],
        [{"node_id": n} for n in sorted(cmpres["onlyA_nodes"])]
    )
    _write_csv(
        os.path.join(out_root, f"nodes_only_{runB}.csv"),
        ["node_id"],
        [{"node_id": n} for n in sorted(cmpres["onlyB_nodes"])]
    )

    # Edges
    _write_csv(
        os.path.join(out_root, "edges_common.csv"),
        ["u","v"],
        [{"u": u, "v": v} for (u,v) in sorted(cmpres["common_edges"])]
    )
    _write_csv(
        os.path.join(out_root, f"edges_only_{runA}.csv"),
        ["u","v"],
        [{"u": u, "v": v} for (u,v) in sorted(cmpres["onlyA_edges"])]
    )
    _write_csv(
        os.path.join(out_root, f"edges_only_{runB}.csv"),
        ["u","v"],
        [{"u": u, "v": v} for (u,v) in sorted(cmpres["onlyB_edges"])]
    )

    # Attribute diffs (flatten)
    def _flatten_attr_diffs(items, node_mode=True):
        flat = []
        if node_mode:
            for it in items:
                nid = it["node_id"]
                for k, (av, bv) in it["diffs"].items():
                    flat.append({"node_id": nid, "attr": k, "A_value": av, "B_value": bv})
        else:
            for it in items:
                (u,v) = it["edge"]
                for k, (av, bv) in it["diffs"].items():
                    flat.append({"u": u, "v": v, "attr": k, "A_value": av, "B_value": bv})
        return flat

    _write_csv(
        os.path.join(out_root, "node_attribute_differences.csv"),
        ["node_id","attr","A_value","B_value"],
        _flatten_attr_diffs(cmpres["node_attr_diffs"], node_mode=True)
    )
    _write_csv(
        os.path.join(out_root, "edge_attribute_differences.csv"),
        ["u","v","attr","A_value","B_value"],
        _flatten_attr_diffs(cmpres["edge_attr_diffs"], node_mode=False)
    )

    # Text report
    report_path = os.path.join(out_root, "comparison_report.txt")
    with open(report_path, "w", encoding="utf-8") as fp:
        fp.write(f"Graph Comparison Report\n")
        fp.write(f"Run A: {runA}\nRun B: {runB}\n")
        fp.write(f"Export root: {root}\nGenerated: {ts}\n\n")

        fp.write("COUNTS\n")
        fp.write(f"- Nodes A: {len(GA['nodes'])}, Nodes B: {len(GB['nodes'])}\n")
        fp.write(f"- Common nodes: {len(cmpres['common_nodes'])}\n")
        fp.write(f"- A-only nodes: {len(cmpres['onlyA_nodes'])}\n")
        fp.write(f"- B-only nodes: {len(cmpres['onlyB_nodes'])}\n\n")
        fp.write(f"- Edges A: {len(GA['edges'])}, Edges B: {len(GB['edges'])}\n")
        fp.write(f"- Common edges: {len(cmpres['common_edges'])}\n")
        fp.write(f"- A-only edges: {len(cmpres['onlyA_edges'])}\n")
        fp.write(f"- B-only edges: {len(cmpres['onlyB_edges'])}\n\n")

        fp.write("ATTRIBUTE DIFFERENCES\n")
        fp.write(f"- Nodes with attribute diffs: {len(cmpres['node_attr_diffs'])}\n")
        fp.write(f"- Edges with attribute diffs: {len(cmpres['edge_attr_diffs'])}\n")

    print(f"[Saved] Comparison CSVs & report at:\n  {out_root}")
    return out_root

def compare_two_exported_runs():
    """Interactive: pick two folders under EXPORT_ROOT and compare them."""
    runs = _list_export_runs(EXPORT_ROOT)
    if not runs:
        print(f"No export runs found under: {EXPORT_ROOT}")
        return

    print("\nAvailable export runs:")
    for i, r in enumerate(runs, 1):
        print(f"  {i}. {r}")

    # Select A
    rawA = input("\nSelect RUN A (number) or 'break' to stop: ").strip()
    if _is_break(rawA):
        print("Stopping by user request.")
        return
    # Select B
    rawB = input("Select RUN B (number) or 'break' to stop: ").strip()
    if _is_break(rawB):
        print("Stopping by user request.")
        return

    try:
        iA = int(rawA); iB = int(rawB)
        assert 1 <= iA <= len(runs) and 1 <= iB <= len(runs) and iA != iB
    except Exception:
        print("Invalid selection. Please run again.")
        return

    runA, runB = runs[iA-1], runs[iB-1]
    folderA = os.path.join(EXPORT_ROOT, runA)
    folderB = os.path.join(EXPORT_ROOT, runB)

    print(f"\n[Loading] RUN A: {folderA}")
    GA = _load_graph_from_folder(folderA)
    print(f"[Loading] RUN B: {folderB}")
    GB = _load_graph_from_folder(folderB)

    print("[Comparing] Computing intersections & differences…")
    cmpres = _compare_graphs(GA, GB)

    # Quick console summary
    print("\n=== SUMMARY ===")
    print(f"Nodes A/B: {len(GA['nodes'])}/{len(GB['nodes'])}")
    print(f"Common nodes: {len(cmpres['common_nodes'])} | A-only: {len(cmpres['onlyA_nodes'])} | B-only: {len(cmpres['onlyB_nodes'])}")
    print(f"Edges A/B: {len(GA['edges'])}/{len(GB['edges'])}")
    print(f"Common edges: {len(cmpres['common_edges'])} | A-only: {len(cmpres['onlyA_edges'])} | B-only: {len(cmpres['onlyB_edges'])}")
    print(f"Node attr diffs: {len(cmpres['node_attr_diffs'])} | Edge attr diffs: {len(cmpres['edge_attr_diffs'])}")

    # Save full artefacts
    out_dir = _save_comparison(EXPORT_ROOT, runA, runB, GA, GB, cmpres)

    # Optional: show a few examples inline
    def _peek(s, n=10): 
        return list(s)[:min(len(s), n)]
    print("\nExamples (first few):")
    print("  A-only nodes:", _peek(cmpres["onlyA_nodes"]))
    print("  B-only nodes:", _peek(cmpres["onlyB_nodes"]))
    print("  A-only edges:", _peek(cmpres["onlyA_edges"]))
    print("  B-only edges:", _peek(cmpres["onlyB_edges"]))
    print("\nDone.")
    return out_dir

# ===================== HOW TO RUN =====================
# Whenever you want to compare two exported runs, call:
# compare_two_exported_runs()
#
# Tip: If you want to run automatically after a fresh export, just call it
# right after Graph.ExportToCSV(...) in your pipeline.
compare_two_exported_runs()
